In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors
import seaborn as sns
import numpy as np

from random import shuffle
from PIL import Image

from pycocotools.coco import COCO

In [ ]:
dataDir='./datasets/path_to_data'
annFile='./datasets/path_to_data'

# Initialize the COCO api for instance annotations
coco=COCO(annFile)

In [ ]:
category_ids = coco.getCatIds()
num_categories = len(category_ids)
print('number of categories: ',num_categories)
for ids in category_ids:
    cats = coco.loadCats(ids=ids)
    print(cats)

In [ ]:
image_ids = coco.getImgIds()
print(len(image_ids))
image_id = image_ids[5]
image_info = coco.loadImgs(image_id)
print(image_info)

In [ ]:
# Load annotations for the given ids
annotation_ids = coco.getAnnIds(imgIds=image_id)
annotations = coco.loadAnns(annotation_ids)

#print(len(annotation_ids))

In [ ]:
filterClasses = ['IC', 'Capacitor', 'Resistor']
catIds = coco.getCatIds(catNms=filterClasses)
print(catIds)

In [ ]:
catID = 2
print(coco.loadCats(ids=catID))

imgId = coco.getImgIds(catIds=[catID])[4]
print(imgId)

In [ ]:
ann_ids = coco.getAnnIds(imgIds=[imgId], iscrowd=None)
print(ann_ids)

In [ ]:
print(f"Annotations for Image ID {imgId}:")
anns = coco.loadAnns(ann_ids)

image_path = dataDir + coco.loadImgs(imgId)[0]['file_name']
print(image_path)
image = plt.imread(image_path)
plt.imshow(image)

coco.showAnns(anns, draw_bbox=True)

plt.axis('off')
plt.title('Annotations for Image ID: {}'.format(image_id))
plt.tight_layout()
plt.show()

In [ ]:
def main():

    cat_ids = coco.getCatIds()
    print(f"Number of Unique Categories: {len(cat_ids)}")
    print("Category IDs:")
    print(cat_ids)  # The IDs are not necessarily consecutive.

    cats = coco.loadCats(cat_ids)
    cat_names = [cat["name"] for cat in cats]
    print("Categories Names:")
    print(cat_names)

    query_id = cat_ids[0]
    query_annotation = coco.loadCats([query_id])[0]
    query_name = query_annotation["name"]
    print("Category ID -> Category Name:")
    print(f"Category ID: {query_id}, Category Name: {query_name}")

    query_name = cat_names[2]
    query_id = coco.getCatIds(catNms=[query_name])[0]
    print("Category Name -> ID:")
    print(f"Category Name: {query_name}, Category ID: {query_id}")

    img_ids = coco.getImgIds(catIds=[query_id])
    print(f"Number of Images Containing {query_name}: {len(img_ids)}")

    img_id = img_ids[2]
    img_info = coco.loadImgs([img_id])[0]
    img_file_name = img_info["file_name"]
    print(f"Image ID: {img_id}, File Name: {img_file_name}")

    ann_ids = coco.getAnnIds(imgIds=[img_id], iscrowd=None)
    anns = coco.loadAnns(ann_ids)
    print(f"Annotations for Image ID {img_id}:")
    print(anns)

    im = plt.imread(dataDir + coco.loadImgs(img_id)[0]['file_name'])
    plt.axis("off")
    plt.imshow(np.asarray(im))
    plt.savefig(f"{img_id}.jpg", bbox_inches="tight", pad_inches=0)
    coco.showAnns(anns, draw_bbox=True)
    plt.savefig(f"{img_id}_annotated.jpg", bbox_inches="tight", pad_inches=0)
    plt.show()
    return

if __name__ == "__main__":

    main()

In [ ]:
catIDs = coco.getCatIds()
cats = coco.loadCats(catIDs)

category_names = [cat['name'].title() for cat in cats]

category_counts = [coco.getImgIds(catIds=[cat['id']]) for cat in cats]
category_counts = [len(img_ids) for img_ids in category_counts]


colors = sns.color_palette('viridis', len(category_names))

plt.figure(figsize=(5, 5))
sns.barplot(x=category_counts, y=category_names, palette=colors)

for i, count in enumerate(category_counts):
    plt.text(count + 20, i, str(count), va='center')
plt.xlabel('Count',fontsize=20)
plt.ylabel('Category',fontsize=20)
plt.title('Category Distribution in PCB Dataset',fontsize=25)
plt.tight_layout()
plt.savefig('coco-cats.png',dpi=300)
plt.show()

In [ ]:
print(category_counts)

In [ ]:
total_count = sum(category_counts)
category_percentages = [(count / total_count) * 100 for count in category_counts]


plt.figure(figsize=(4, 4))


labels = [f"{name} " for name, percentage in zip(category_names, category_percentages)]
label_props = {"fontsize": 25, 
               "bbox": {"edgecolor": "white", 
                        "facecolor": "white", 
                        "alpha": 0.7, 
                        "pad": 0.5}
              }

wedges, _, autotexts = plt.pie(category_counts, 
                              autopct='', 
                              startangle=90, 
                              textprops=label_props, 
                              pctdistance=0.85)

legend_labels = [f"{label}\n{category_percentages[i]:.1f}%" for i, label in enumerate(labels)]
plt.legend(wedges, legend_labels, title="Categories", loc="upper center", bbox_to_anchor=(0.5, -0.01), 
           ncol=4, fontsize=12)

plt.axis('equal')
plt.title('Category Distribution in COCO Dataset', fontsize=29)
plt.tight_layout()
plt.savefig('coco-dis.png', dpi=300)
plt.show()

In [ ]:
total_count # the above analysis is wrong. somewhere data was misread or labeled.

In [ ]:
import sklearn
import funcy
import argparse

In [ ]:
#pip install scikit-multilearn

In [ ]:
!python cocosplit.py -s 0.7 result.json train.json test.json

In [ ]:
from tensorflow import keras

ANNOTATION_FILE_TRAIN = 'data/train.json'
ANNOTATION_FILE_VAL = 'data/test.json'

coco_train = COCO(ANNOTATION_FILE_TRAIN)
catIds_train = coco_train.getCatIds()
imgIds_train = coco_train.getImgIds()
imgDict_train = coco_train.loadImgs(imgIds_train)

coco_val = COCO(ANNOTATION_FILE_VAL)
catIds_val = coco_val.getCatIds()
imgIds_val = coco_val.getImgIds()
imgDict_val = coco_val.loadImgs(imgIds_val)

print(len(imgIds_train), len(catIds_train))
print(len(imgIds_val), len(catIds_val))
print(catIds_val)
print(coco_val.getImgIds())
print(coco_train.getImgIds())

In [ ]:
os.getcwd()

In [ ]:
import os
import json
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models, optimizers
from tqdm import tqdm

### using ADAM model --> terrible results / needs debugging

def load_coco_dataset(coco_annotations_file, image_dir):
    with open(coco_annotations_file, 'r') as f:
        coco_data = json.load(f)

    images = coco_data['images']
    annotations = coco_data['annotations']

    X = []
    y = []

    for image in tqdm(images, desc="Loading images"):
        image_id = image['id']
        img_path = os.path.join(image_dir, image['file_name'])
        category_id = [annotation['category_id'] for annotation in annotations if annotation['image_id'] == image_id]
        if category_id:  # Ensure image has at least one annotation
            X.append(img_path)
            y.append(category_id[0])  # Take the first category as the label

    return np.array(X), np.array(y)

def preprocess_images(X):
    X_processed = []
    for img_path in tqdm(X, desc="Preprocessing images"):
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array /= 255.0  # Normalize pixel values
        X_processed.append(img_array)
    return np.array(X_processed)

COCO_ANNOTATIONS_FILE = 'result.json'
IMAGE_DIR = './datasets/path_to_images'
X, y = load_coco_dataset(COCO_ANNOTATIONS_FILE, IMAGE_DIR)

X = preprocess_images(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(80, activation='softmax')  # Assuming 80 COCO classes
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes))

In [ ]:
X_train.shape # (number samples, height, width, channels (RBG)

In [ ]:
X_test.shape

In [ ]:
y_train.shape # only holds labels

In [ ]:
y_test.shape

In [ ]:
print(y_test)

In [ ]:
from torchvision.models.detection import (
    fasterrcnn_mobilenet_v3_large_fpn,
    FasterRCNN_MobileNet_V3_Large_FPN_Weights,
)

In [ ]:
import torch

In [ ]:
NUM_CLASSES = 3

def get_faster_rcnn_model(num_classes):
    """return model and preprocessing transform"""
    model = fasterrcnn_mobilenet_v3_large_fpn(
        weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT
    )
    model.roi_heads.box_predictor.cls_score = torch.nn.Linear(
        in_features=model.roi_heads.box_predictor.cls_score.in_features,
        out_features=num_classes,
        bias=True,
    )
    model.roi_heads.box_predictor.bbox_pred = torch.nn.Linear(
        in_features=model.roi_heads.box_predictor.bbox_pred.in_features,
        out_features=num_classes * 4,
        bias=True,
    )
    preprocess = FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT.transforms()
    return model, preprocess


model, preprocess = get_faster_rcnn_model(num_classes=NUM_CLASSES)

In [ ]:
print(model.transform)

In [ ]:
import json
from collections import defaultdict
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset


class CocoDataset(Dataset):
    """PyTorch dataset for COCO annotations."""

    # adapted from https://github.com/pytorch/vision/issues/2720

    def __init__(self, root, annFile, transform=None):
        """Load COCO annotation data."""
        self.data_dir = Path(root)
        self.transform = transform

        # load the COCO annotations json
        anno_file_path = annFile
        with open(str(anno_file_path)) as file_obj:
            self.coco_data = json.load(file_obj)
        # put all of the annos into a dict where keys are image IDs to speed up retrieval
        self.image_id_to_annos = defaultdict(list)
        for anno in self.coco_data["annotations"]:
            image_id = anno["image_id"]
            self.image_id_to_annos[image_id] += [anno]

    def __len__(self):
        return len(self.coco_data["images"])

    def __getitem__(self, index):
        """Return tuple of image and labels as torch tensors."""
        image_data = self.coco_data["images"][index]
        image_id = image_data["id"]
        image_path = self.data_dir / image_data["file_name"]
        image = Image.open(image_path).convert("RGB")

        annos = self.image_id_to_annos[image_id]
        anno_data = {
            "boxes": [],
            "labels": [],
            "area": [],
            "iscrowd": [],
        }
        for anno in annos:
            coco_bbox = anno["bbox"]
            left = coco_bbox[0]
            top = coco_bbox[1]
            right = coco_bbox[0] + coco_bbox[2]
            bottom = coco_bbox[1] + coco_bbox[3]
            area = coco_bbox[2] * coco_bbox[3]
            anno_data["boxes"].append([left, top, right, bottom])
            anno_data["labels"].append(anno["category_id"])
            anno_data["area"].append(area)
            anno_data["iscrowd"].append(anno["iscrowd"])

        target = {
            "boxes": torch.as_tensor(anno_data["boxes"], dtype=torch.float32),
            "labels": torch.as_tensor(anno_data["labels"], dtype=torch.int64),
            "image_id": torch.tensor([image_id]),
            "area": torch.as_tensor(anno_data["area"], dtype=torch.float32),
            "iscrowd": torch.as_tensor(anno_data["iscrowd"], dtype=torch.int64),
        }

        if self.transform is not None:
            image = self.transform(image)

        return image, target

In [ ]:
import random
import torchvision.transforms as T
from IPython.display import display
from PIL import ImageDraw

# create datasets
training_dataset = CocoDataset(
    root="./",
    annFile="test.json",
    transform=preprocess,
)
validation_dataset = CocoDataset(
    root="./",
    annFile="train.json",
    transform=preprocess,
)

print(f"training dataset size: {training_dataset.__len__()}")

print(f"validation dataset size: {validation_dataset.__len__()}")

In [ ]:
img, label = training_dataset[random.randint(0, len(training_dataset) - 1)]
print(f"random training label: {label}")

transform = T.ToPILImage()
img = transform(img)
x1, y1, x2, y2 = label["boxes"].numpy()[0]
draw = ImageDraw.Draw(img)
draw.rectangle([x1, y1, x2, y2], fill=None, outline="#ff0000cc", width=2)
display(img)

In [ ]:
BATCH_SIZE = 1

def collate(batch):
    """return tuple data"""
    return tuple(zip(*batch))

train_loader = torch.utils.data.DataLoader(
    training_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate,
)

validation_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate,
)

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]

In [ ]:
#print(params)

In [ ]:
optimizer = torch.optim.SGD(
    params, 
    lr=0.001, 
    momentum=0.9, 
    weight_decay=0.0005
)

In [ ]:
#for images, targets in train_loader:
#    print(images, targets)

In [ ]:
num_epochs = 20
train_loss_list = []
validation_loss_list = []
model.train()
for epoch in range(num_epochs):
    N = len(train_loader.dataset)
    current_train_loss = 0
    # train loop
    for images, targets in train_loader:
#        images = list(image.to(device) for image in images)
#        targets = [
#            {
#                k: v.to(device) if isinstance(v, torch.Tensor) else v
#                for k, v in t.items()
#            }
#            for t in targets
#        ]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        current_train_loss += losses
    train_loss_list.append(current_train_loss / N)

    # validation loop
    N = len(validation_loader.dataset)
    current_validation_loss = 0
    with torch.no_grad():
#        for images, targets in validation_loader:
#            images = list(image.to(device) for image in images)
#            targets = [
#                {
#                    k: v.to(device) if isinstance(v, torch.Tensor) else v
#                    for k, v in t.items()
#                }
#                for t in targets
#            ]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            current_validation_loss += losses
    validation_loss_list.append(current_validation_loss / N)

    print(f"epoch: {epoch}")
    print(
        f"train loss: {train_loss_list[-1]}, validation loss: {validation_loss_list[-1]}"
    )

In [ ]:
torch.save(model, "./model.pth") # save model to file

# plot losses
train_loss = [x.cpu().detach().numpy() for x in train_loss_list]
validation_loss = [x.cpu().detach().numpy() for x in validation_loss_list]

plt.plot(train_loss, "-o", label="train loss")
plt.plot(validation_loss, "-o", label="validation loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()

In [ ]:
model = torch.load("./model.pth")

def inference(img, model):
    model.eval()
    pred = model([img]) # forward pass

    transform = T.ToPILImage()
    img = transform(img)
    x1, y1, x2, y2 = pred[0]["boxes"].cpu().detach().numpy()[0]
    draw = ImageDraw.Draw(img)
    draw.rectangle([x1, y1, x2, y2], fill=None, outline="#ff0000cc", width=2)
    display(img)
    return pred

In [ ]:
img, _ = validation_dataset[random.randint(0, len(validation_dataset) - 1)]

In [ ]:
inference(img, model)

In [ ]:
#labels = ['Capacitor', 'IC', 'Resistor']
#labels = [0, 1, 2]

In [ ]:
import matplotlib.pyplot as plt

categories = ['Capacitors', 'Resistors', 'Integrated Circuits']
values = [149, 133, 24]

plt.figure(figsize=(8, 6))
plt.pie(values, labels=categories, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Component Annotations for PCB Dataset')
plt.axis('equal') 
plt.show()